In [12]:
import nltk
import random

nltk.download('stopwords')

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import pandas as pd
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bharathkumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
tweets = pd.read_csv("final_tweets_list.csv")

In [15]:
tweets

,Unnamed: 0,event_id,tweets
0,0,Georgia1,https://t.co/KwMLUjBum9\n\nThree jail guards i...
1,1,Georgia1,September Complaint: Voting machine testing co...
2,2,Georgia1,A man accused of fatally shooting another pers...
3,3,Georgia1,"Oops…\n\nGA Tech’s defense has been okay, but ..."
4,4,Georgia1,Scott Frost bragged about how his new O-Line c...
...,...,...,...
2235,179,Washington14,@Madonna I make intercessions with Heaven for ...
2236,180,Washington14,Remember when Eagles coach Nick Sirianni took ...
2237,181,Washington14,@SovCam @juliegreen52722 @GloryBound_info It w...
2238,182,Washington14,@NWSportsfan1992 @Softykjr These polls are sup...


In [25]:
stopwords_english = stopwords.words('english')
tokenizer = TweetTokenizer(preserve_case=False,
                           strip_handles=True,
                           reduce_len=True)
stemmer = PorterStemmer()
link_removed_text = []
before_stemming_list = []
after_stemming_list = []
for tweet in tweets["tweets"].tolist():
    # print('Original Tweet: ')
    # print(a)

    # it will remove the old style retweet text "RT"
    tweet2 = re.sub(r'^RT[\s]+', '', tweet)

    # it will remove hyperlinks
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)


    # it will remove hashtags. We have to be careful here not to remove
    # the whole hashtag because text of hashtags contains huge information.
    # only removing the hash # sign from the word
    tweet2 = re.sub(r'#', '', tweet2)

    # it will remove single numeric terms in the tweet.
    tweet2 = re.sub(r'[0-9]', '', tweet2)
    tweet2 = tweet2.replace("’", "")
    tweet2 = tweet2.replace("`", "")

    # print('\nAfter removing old style tweet, hyperlinks and # sign')
    # print(tweet2)
    link_removed_text.append(tweet2)

    tweet_tokens = tokenizer.tokenize(tweet2)

    tweets_clean = []
    for word in tweet_tokens: # Go through every word in your tokens list
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
    before_stemming_list.append(' '.join(tweets_clean))

In [26]:
tweets["cleaned"] = before_stemming_list

In [28]:
tweets_based_on_event_id = tweets.groupby('event_id')["cleaned"].apply(list)

In [29]:
tweets_based_on_event_id

event_id
Boston1      [need watch watched flight boston september re...
Boston2      [cfp decolonizations discontents dissent oppos...
Boston3      [contact rhode island-based colleen creativegr...
Boston4      [went back home boston another ultra-liberal c...
Boston5      [“ bunch freaks honest ” ― kakkz photographer ...
                                   ...                        
illinois5    [compare deaths september illinois alone imagi...
illinois6    [cofa updated website fy estimated revenue col...
illinois7    [true see nin riot fest illinois september lik...
illinois8    [like marilyn manson lol tour together though ...
illinois9    [sure wanna go back arguing crazy left instead...
Name: cleaned, Length: 163, dtype: object

In [32]:
keys = list(tweets_based_on_event_id.keys())

In [34]:
combined_tweets = []
for key in keys:
    strr = ' '.join(tweets_based_on_event_id.get(key))
    combined_tweets.append(strr)

In [36]:
final_pre_processed_tweets_data = pd.DataFrame()
final_pre_processed_tweets_data["event_id"] = keys
final_pre_processed_tweets_data["combined_tweets"] = combined_tweets

In [38]:
final_pre_processed_tweets_data.to_csv("tweets_for_cassification.csv", index=False)

In [39]:
classified_data = pd.read_csv("classified_events.csv")

In [40]:
classified_data.columns

Index(['Unnamed: 0', 'event_id', 'combined_tweets', 'classification'], dtype='object')

In [42]:
classified_data = classified_data.drop(['Unnamed: 0', 'combined_tweets'], axis=1)

In [43]:
classified_data

,event_id,classification
0,Boston1,entertainment
1,Boston2,sport
2,Boston3,sport
3,Boston4,sport
4,Boston5,entertainment
...,...,...
158,illinois5,business
159,illinois6,others
160,illinois7,others
161,illinois8,business


In [45]:
final_org_data = pd.read_csv("final_dataset_v1.csv")
final_org_data

,Unnamed: 0,event_id,location,date,tweets_count,keywords,event_sentiment
0,0,Washington1,Washington,9/13/22,13,"[('allies', 0.0370141109748817), ('trafalgar',...",Positive
1,1,Washington2,Washington,9/16/22,13,"[('corsi', 0.04162912505109767), ('interview',...",Negative
2,2,Washington3,Washington,9/3/22,8,"[('director', 0.10912854635271647), ('curiousl...",Positive
3,3,Washington4,Washington,9/9/22,14,"[('maryland', 0.02620209941119737), ('boulevar...",Negative
4,4,Washington5,Washington,9/10/22,27,"[('george', 0.04167087239681818), ('school', 0...",Positive
...,...,...,...,...,...,...,...
139,2,Georgia3,Georgia,9/10/22,12,"[('county', 0.021428847152118378), ('camden', ...",Negative
140,3,Georgia4,Georgia,9/6/22,13,"[('county', 0.032262332786760446), ('sheriff',...",Negative
141,4,Georgia5,Georgia,9/13/22,5,"[('israeli', 0.06771232806350562), ('businessm...",Positive
142,5,Georgia6,Georgia,9/16/22,14,"[('mark', 0.06777092907666397), ('sabbath', 0....",Negative


In [46]:
new_df_df = pd.merge(final_org_data, classified_data, on="event_id")

In [50]:
new_df_df.to_csv("final_dataset_v1.csv", index=False)